# Tool practice
* tool 사용 이유: 외부 기능을 호출할 수 있게 됨.  
-> Agent가 필요시 tool 호출-> llm에게 전달  
-> 외부 정보를 반영한 답변 생성  

환경  
* 사용 API: 공공데이터 API 
* llm model: llama3.1:8b

In [2]:
import requests
from collections import defaultdict
from datetime import datetime
import os
from langchain.tools import tool

def get_weather_info(base_date, base_time, nx, ny):
    """
    날씨 정보를 가져오는 tool
    반환: {(fcstDate, fcstTime): {category: fcstValue, ...}, ...}
    ex. input 데이터 예시 
    base_date = "20250806", "base_time": "0500", "nx": "55", "ny": "127"
    """

    service_key = 'fZH6JY/kjWMLbFXVG2QsHmWb0zTllX8n6+RSpHawkKhGR+lhFSvkiyJgZDnhvMLHIQJnOsmjqTL2zXq7UUiCGw=='
    key = service_key or os.getenv("KMA_SERVICE_KEY")
    if not key:
        raise ValueError("service_key가 필요합니다.")
    
    params = {
    "serviceKey": key,
    "numOfRows": "60",
    "pageNo": "1",
    "dataType": "JSON",
    "base_date": base_date,
    "base_time": base_time,
    "nx": nx,
    "ny": ny
    }
    
    try:
        response = requests.get("http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst", params=params).json()
        data = response['response']['body']['items']['item']

        if not data:
            return "검색 결과가 없습니다."

        # 오늘 날짜 기준 설정
        today = datetime.today()
        
        # 날짜/시간 단위로 category 묶기
        grouped = defaultdict(dict)
        for item in data:
            key = (item['fcstDate'], item['fcstTime'])
            grouped[key][item['category']] = item['fcstValue']

        # 보기 좋게 출력
        for (fcstDate, fcstTime), categories in sorted(grouped.items()):
            target_day = datetime.strptime(fcstDate, "%Y%m%d")
            days_after = (target_day - today).days

            print(f"\n📅 {fcstDate} {fcstTime[:2]}시 예보 ({days_after}일 후):")
            print(f"  🌡️ 기온(TMP): {categories.get('TMP', '-')}℃")
            print(f"  🌬️ 동서풍(UUU): {categories.get('UUU', '-')}")
            print(f"  🌬️ 남북풍(VVV): {categories.get('VVV', '-')}")
            print(f"  🌧️ 강수형태(PTY): {categories.get('PTY', '-')}")
            print(f"  ☁️ 하늘상태(SKY): {categories.get('SKY', '-')}")
            print(f"  ☔ 강수확률(POP): {categories.get('POP', '-')}")

    except requests.RequestException as e:
        return f"검색 요청 중 오류가 발생했습니다: {e}"

In [3]:
# 함수 호출
base_date = "20250809"
base_time= "0500"
nx= "55" 
ny = "127"
get_weather_info(base_date, base_time, nx, ny)


📅 20250809 06시 예보 (-1일 후):
  🌡️ 기온(TMP): 24℃
  🌬️ 동서풍(UUU): -0.7
  🌬️ 남북풍(VVV): 0.6
  🌧️ 강수형태(PTY): 0
  ☁️ 하늘상태(SKY): 4
  ☔ 강수확률(POP): 30

📅 20250809 07시 예보 (-1일 후):
  🌡️ 기온(TMP): 24℃
  🌬️ 동서풍(UUU): -0.9
  🌬️ 남북풍(VVV): 0.9
  🌧️ 강수형태(PTY): 0
  ☁️ 하늘상태(SKY): 4
  ☔ 강수확률(POP): 30

📅 20250809 08시 예보 (-1일 후):
  🌡️ 기온(TMP): 25℃
  🌬️ 동서풍(UUU): -0.5
  🌬️ 남북풍(VVV): 1.6
  🌧️ 강수형태(PTY): 0
  ☁️ 하늘상태(SKY): 4
  ☔ 강수확률(POP): 30

📅 20250809 09시 예보 (-1일 후):
  🌡️ 기온(TMP): 26℃
  🌬️ 동서풍(UUU): -0.4
  🌬️ 남북풍(VVV): 1.5
  🌧️ 강수형태(PTY): 0
  ☁️ 하늘상태(SKY): 4
  ☔ 강수확률(POP): 30

📅 20250809 10시 예보 (-1일 후):
  🌡️ 기온(TMP): 26℃
  🌬️ 동서풍(UUU): 0.3
  🌬️ 남북풍(VVV): 1.5
  🌧️ 강수형태(PTY): 0
  ☁️ 하늘상태(SKY): 4
  ☔ 강수확률(POP): 30


# 간단한 tool 출력

In [4]:
# infer_schema= True: Pydantic 모델을 지정 없이 자동 스키마 생성
# return_direct=False: 에이전트가 툴 호출시 결과를 llm이 받아서 추가 설명 가공
@tool("get_weather_info", infer_schema=True, return_direct=True)
def get_weather_info(base_date, base_time, nx, ny):
    """
    날씨 정보를 가져오는 tool
    반환: {(fcstDate, fcstTime): {category: fcstValue, ...}, ...}
    ex. input 데이터 예시 
    base_date = "20250806", "base_time": "0500", "nx": "55", "ny": "127"
    """

    service_key = 'fZH6JY/kjWMLbFXVG2QsHmWb0zTllX8n6+RSpHawkKhGR+lhFSvkiyJgZDnhvMLHIQJnOsmjqTL2zXq7UUiCGw=='
    key = service_key or os.getenv("KMA_SERVICE_KEY")
    if not key:
        raise ValueError("service_key가 필요합니다.")
    
    params = {
    "serviceKey": key,
    "numOfRows": "60",
    "pageNo": "1",
    "dataType": "JSON",
    "base_date": base_date,
    "base_time": base_time,
    "nx": nx,
    "ny": ny
    }
    
    try:
        response = requests.get("http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst", params=params).json()
        data = response['response']['body']['items']['item']

        if not data:
            return "검색 결과가 없습니다."

        # 오늘 날짜 기준 설정
        today = datetime.today()
        
        # 날짜/시간 단위로 category 묶기
        grouped = defaultdict(dict)
        for item in data:
            key = (item['fcstDate'], item['fcstTime'])
            grouped[key][item['category']] = item['fcstValue']

        # 보기 좋게 출력
        for (fcstDate, fcstTime), categories in sorted(grouped.items()):
            target_day = datetime.strptime(fcstDate, "%Y%m%d")
            days_after = (target_day - today).days

            print(f"\n📅 {fcstDate} {fcstTime[:2]}시 예보 ({days_after}일 후):")
            print(f"  🌡️ 기온(TMP): {categories.get('TMP', '-')}℃")
            print(f"  🌬️ 동서풍(UUU): {categories.get('UUU', '-')}")
            print(f"  🌬️ 남북풍(VVV): {categories.get('VVV', '-')}")
            print(f"  🌧️ 강수형태(PTY): {categories.get('PTY', '-')}")
            print(f"  ☁️ 하늘상태(SKY): {categories.get('SKY', '-')}")
            print(f"  ☔ 강수확률(POP): {categories.get('POP', '-')}")

    except requests.RequestException as e:
        return f"검색 요청 중 오류가 발생했습니다: {e}"

In [5]:
out = get_weather_info.invoke({
        "base_date": "20250809",
        "base_time": "0500",
        "nx": "55",
        "ny": "127",
    })
print(out)


📅 20250809 06시 예보 (-1일 후):
  🌡️ 기온(TMP): 24℃
  🌬️ 동서풍(UUU): -0.7
  🌬️ 남북풍(VVV): 0.6
  🌧️ 강수형태(PTY): 0
  ☁️ 하늘상태(SKY): 4
  ☔ 강수확률(POP): 30

📅 20250809 07시 예보 (-1일 후):
  🌡️ 기온(TMP): 24℃
  🌬️ 동서풍(UUU): -0.9
  🌬️ 남북풍(VVV): 0.9
  🌧️ 강수형태(PTY): 0
  ☁️ 하늘상태(SKY): 4
  ☔ 강수확률(POP): 30

📅 20250809 08시 예보 (-1일 후):
  🌡️ 기온(TMP): 25℃
  🌬️ 동서풍(UUU): -0.5
  🌬️ 남북풍(VVV): 1.6
  🌧️ 강수형태(PTY): 0
  ☁️ 하늘상태(SKY): 4
  ☔ 강수확률(POP): 30

📅 20250809 09시 예보 (-1일 후):
  🌡️ 기온(TMP): 26℃
  🌬️ 동서풍(UUU): -0.4
  🌬️ 남북풍(VVV): 1.5
  🌧️ 강수형태(PTY): 0
  ☁️ 하늘상태(SKY): 4
  ☔ 강수확률(POP): 30

📅 20250809 10시 예보 (-1일 후):
  🌡️ 기온(TMP): 26℃
  🌬️ 동서풍(UUU): 0.3
  🌬️ 남북풍(VVV): 1.5
  🌧️ 강수형태(PTY): 0
  ☁️ 하늘상태(SKY): 4
  ☔ 강수확률(POP): 30
None


In [ ]:
# 함수로 호출시 에러 발생
base_date = "20250809"
base_time= "0500"
nx= "55" 
ny = "127"
get_weather_info(base_date, base_time, nx, ny)

C:\Users\21ckw\AppData\Local\Temp\ipykernel_1520\185682419.py:6: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  get_weather_info(base_date, base_time, nx, ny)


TypeError: BaseTool.__call__() takes from 2 to 3 positional arguments but 5 were given